In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pickle
# importin the necessary.
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
dataframe = pd.read_csv('Data/Churn_Modelling.csv')

In [ ]:
dataframe.head()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(dataframe.corr())

In [ ]:
dataframe.info()

In [ ]:
dataframe.nunique()

In [ ]:
# Removing unwanted data
dataframe.drop(['RowNumber','CustomerId','Surname'],inplace=True,axis=1)

In [ ]:
# dataframe['Gender']= le.fit_transform(dataframe['Gender'])

In [ ]:
dataframe.shape

In [ ]:
dataframe.isna().sum()

In [ ]:
plt.figure(figsize=(10,10))
dataframe.boxplot(rot=45)

In [ ]:
dataframe.hist(figsize=(10,10))

In [ ]:
gender_label_encoder = LabelBinarizer()

In [ ]:
gender_label_encoder.fit(dataframe['Gender'])

In [ ]:
dataframe['Gender']=gender_label_encoder.transform(dataframe['Gender'])

In [ ]:
with open('pickle/label_binarize_gender.pkl','wb') as f:
    pickle.dump(gender_label_encoder,f)

In [ ]:
train,test = train_test_split(dataframe,test_size=0.1)
test.to_csv('Data/test.csv')

In [ ]:
class One_Hot_encoder():
        
    def One_hot_encoder(self,dataframe,col,lb):
        lb.fit(dataframe[col])
        b = lb.transform(dataframe[col])
        names = dataframe[col].unique()
        for i in range(0,len(names)-1):
            dataframe[names[i]]=b[:,i]
        with open('pickle/label_binarizer_'+col+'.pkl','wb') as f:
            pickle.dump(lb,f)
        return dataframe.drop([col],axis=1)
    
    def sep(self,dataframe,col):
        return dataframe.drop([col],axis=1),dataframe[col]

In [ ]:
ohe = One_Hot_encoder()
train = ohe.One_hot_encoder(train,'Geography',LabelBinarizer())

In [ ]:
# dataframe

In [ ]:
with open('Data/One_Hot_encoder_class_object.pkl','wb') as f:
    pickle.dump(ohe,f)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(train.corr())

In [ ]:
train = train.reset_index()
test=test.reset_index()
try:
    train = train.drop(['index'],axis=1)
except Exception as e:
    pass
try:
    test = test.drop(['index'],axis=1)
except Exception as e:
    pass

In [ ]:
train_x,train_y = ohe.sep(train,'Exited')

In [ ]:
ss = StandardScaler()
ss.fit(train_x)
b = ss.transform(train_x)

In [ ]:
with open('pickle/scaler.pkl','wb') as f:
    pickle.dump(ss,f)

In [ ]:
# WE CAN INITIALIZE our neural network by 2 ways:
# either by defiining the sequence of layers or a graph
# AS we'll be defining sequence of layers hence we initialize as sequnce
clasifier = Sequential()

In [ ]:
# Adding input layer
# out_dim = nuber of nodes in hidden layer as output is 0 or 1 hence only 1 layer in output and as we have 11 features 1+11/2=6
# We can decide it any number as we can choose any number of nodes
# init it is a fucntion which initializes  uniform keeps small random weights close to 0
# activation function
# number of input nodes in input layer, and we have 11 independent variables. 
# Dence because: 
clasifier.add(Dense(output_dim=6, init='uniform', activation='relu',input_dim=11))
# second layer again output 11+1/2 
clasifier.add(Dense(output_dim=6, init='uniform', activation='relu'))
# output layers 
# output node is 1 as here need only 1 output
clasifier.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

# running neural network
# optimiser : algo to optimise weights: 1 of stochastic grad algo is adam  
# loss: sum of sq of diff 
clasifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# number of epochs 
clasifier.fit(np.array(train_x),np.array(train_y).reshape(-1,1), batch_size=10, nb_epoch = 100)

In [ ]:
print("accuracy using {}".format(accuracy_score(test_y,y_pred),f1_score())
cm = confusion_matrix(test_y,y_pred)

In [ ]:
with open('pickle/classifier.pkl','wb') as f:
    pickle.dump(clasifier,f)